In [1]:
# download nltk stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
# Install a particular version of `google-cloud-storage` because (oddly enough)
# the  version on Colab and GCP is old. A dependency error below is okay.
!pip install -q google-cloud-storage==1.43.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.6/106.6 kB 1.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 0.22.0 requires google-cloud-storage>=2.0.0, but you have google-cloud-storage 1.43.0 which is incompatible.


In [3]:
# authenticate below for Google Storage access as needed
from google.colab import auth
auth.authenticate_user()

In [4]:
!pip install pyngrok

In [5]:
# TODO: sign up for an ngrok account
# then put your ngrok token below and execute
from pyngrok import ngrok
# in ngrok.yml
# authtoken: 2dKEqAWUID2C3fhYjzA9tIp4mCB_5jr2vvRv53XzDkTTkMrRj
ngrok.set_auth_token("2dKEqAWUID2C3fhYjzA9tIp4mCB_5jr2vvRv53XzDkTTkMrRj")

# Run the app

In [6]:
# you need to upload your implementation of search_app.py
import search_frontend as se

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# uncomment the code below and execute to reload the module when you make
# changes to search_frontend.py (after you upload again).
# import importlib
# importlib.reload(se)

In [7]:
public_url = ngrok.connect("5000").public_url
print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:5000\"")
# Update any base URLs to use the public ngrok URL
se.app.config["BASE_URL"] = public_url
se.app.run()

 * ngrok tunnel "https://8104-34-71-121-173.ngrok-free.app" -> "http://127.0.0.1:5000"
 * Serving Flask app 'search_frontend'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


# Testing your app

Once your app is running you can query it. You can simply do that by navigating to the URL that ngrok gave you above or through code in a different python session. For example, once the frontend app is running, you can navigate to:
http://YOUR_SERVER_DOMAIN/search?query=hello+world where YOUR_SERVER_DOMAIN is something like XXXX-XX-XX-XX-XX.ngrok.io, which is printed above in Colab or that is your external IP on GCP.

The code below shows how to issue a query from python. This is also how our testing code will issue queries to your search engine, so make sure to test your search engine this way after you deploy it to GCP and before submission. Command line instructions for deploying your search engine to GCP are available at `run_frontend_in_gcp.sh`. Note that we will not only issue training queries to your search engine, but also test queries, i.e. queries that you've never seen before.

In [8]:
import json
import json
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score

with open('queries_train.json', 'rt') as f:
  queries = json.load(f)

In [9]:
def average_precision(true_list, predicted_list, k=40):
    true_set = frozenset(true_list)
    predicted_list = predicted_list[:k]
    precisions = []
    for i,doc_id in enumerate(predicted_list):
        if doc_id in true_set:
            prec = (len(precisions)+1) / (i+1)
            precisions.append(prec)
    if len(precisions) == 0:
        return 0.0
    return round(sum(precisions)/len(precisions),3)

In [10]:
def precision_at_k(true_list, predicted_list, k):
    true_set = frozenset(true_list)
    predicted_list = predicted_list[:k]
    if len(predicted_list) == 0:
        return 0.0
    return round(len([1 for doc_id in predicted_list if doc_id in true_set]) / len(predicted_list), 3)
def recall_at_k(true_list, predicted_list, k):
    true_set = frozenset(true_list)
    predicted_list = predicted_list[:k]
    if len(true_set) < 1:
        return 1.0
    return round(len([1 for doc_id in predicted_list if doc_id in true_set]) / len(true_set), 3)
def f1_at_k(true_list, predicted_list, k):
    p = precision_at_k(true_list, predicted_list, k)
    r = recall_at_k(true_list, predicted_list, k)
    if p == 0.0 or r == 0.0:
        return 0.0
    return round(2.0 / (1.0/p + 1.0/r), 3)
def results_quality(true_list, predicted_list):
    p5 = precision_at_k(true_list, predicted_list, 5)
    f1_30 = f1_at_k(true_list, predicted_list, 30)
    if p5 == 0.0 or f1_30 == 0.0:
        return 0.0
    return round(2.0 / (1.0/p5 + 1.0/f1_30), 3)

assert precision_at_k(range(10), [1,2,3] , 2) == 1.0
assert recall_at_k(   range(10), [10,5,3], 2) == 0.1
assert precision_at_k(range(10), []      , 2) == 0.0
assert precision_at_k([],        [1,2,3],  5) == 0.0
assert recall_at_k(   [],        [10,5,3], 2) == 1.0
assert recall_at_k(   range(10), [],       2) == 0.0
assert f1_at_k(       [],        [1,2,3],  5) == 0.0
assert f1_at_k(       range(10), [],       2) == 0.0
assert f1_at_k(       range(10), [0,1,2],  2) == 0.333
assert f1_at_k(       range(50), range(5), 30) == 0.182
assert f1_at_k(       range(50), range(10), 30) == 0.333
assert f1_at_k(       range(50), range(30), 30) == 0.75
assert results_quality(range(50), range(5))  == 0.308
assert results_quality(range(50), range(10)) == 0.5
assert results_quality(range(50), range(30)) == 0.857
assert results_quality(range(50), [-1]*5 + list(range(5,30))) == 0.0


In [11]:
from time import time as current_time  # Rename the time variable
import requests

url = 'https://api.ngrok.com/reserved_domains/rd_2dKKBcXh0yNuhkFPROUvdYUUE8c'

# Set up 10-fold cross-validation
# Set up 10-fold cross-validation
num_folds = 10
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

# Initialize a list to store the metrics for each fold
fold_metrics = []

# Iterate through the folds
for fold, (train_index, test_index) in enumerate(kf.split(queries)):
    train_queries = [list(queries.keys())[i] for i in train_index]
    test_queries = [list(queries.keys())[i] for i in test_index]

    # Your training code here (if needed)

    # Your testing code here
    fold_results = []
    for q in test_queries:
        duration, rq = None, None
        t_start = current_time()
        try:
            res = requests.get(url + '/search', {'query': q}, timeout=35)
            duration = current_time() - t_start
            if res.status_code == 200:
                pred_wids, _ = zip(*res.json())

                # For this scenario, just print the retrieved document IDs without comparison
                print(f"Query: {q}, Retrieved Document IDs: {pred_wids}")

                # You can still calculate and store other metrics if needed
                # Calculate metrics for the current query using true_wids and pred_wids
                precision = precision_at_k(true_wids, pred_wids, 5)
                recall = recall_at_k(true_wids, pred_wids, 5)
                f1 = f1_at_k(true_wids, pred_wids, 5)
                quality = results_quality(true_wids, pred_wids)
                fold_results.append({'query': q, 'duration': duration, 'precision': precision, 'recall': recall, 'f1': f1, 'quality': quality})
        except Exception as e:
            print(f"Error processing query '{q}' in Fold {fold + 1}: {e}")

    fold_metrics.append(fold_results)

# Save the metrics for each fold to a dictionary
metrics_dict = {f"Fold {i + 1}": fold_results for i, fold_results in enumerate(fold_metrics)}

# Print the metrics for each fold
for fold, metrics in metrics_dict.items():
    print(f"\n{fold} Metrics:")
    for query_metrics in metrics:
        print(f"  Query: {query_metrics['query']}, Duration: {query_metrics['duration']}s, Metrics: {query_metrics}")

# Calculate and print the average metrics across all folds
average_metrics = {}
for metric_name in ['precision', 'recall', 'f1', 'quality']:
    metric_values = [query_metrics[metric_name] for fold_results in metrics_dict.values() for query_metrics in fold_results]

    # Check if the metric_values list is empty
    if len(metric_values) > 0:
        average_metric_value = sum(metric_values) / len(metric_values)
    else:
        average_metric_value = 0.0

    average_metrics[metric_name] = average_metric_value

print("\nAverage Metrics Across Folds:")
for metric_name, value in average_metrics.items():
    print(f"  {metric_name.capitalize()}: {value}")


Fold 1 Metrics:

Fold 2 Metrics:

Fold 3 Metrics:

Fold 4 Metrics:

Fold 5 Metrics:

Fold 6 Metrics:

Fold 7 Metrics:

Fold 8 Metrics:

Fold 9 Metrics:

Fold 10 Metrics:

Average Metrics Across Folds:
  Precision: 0.0
  Recall: 0.0
  F1: 0.0
  Quality: 0.0


In [14]:
print(queries)

{'genetics': ['12266', '8406655', '1590357', '4250553', '339838', '7955', '219268', '19595', '12383', '12385', '1763082', '159266', '274192', '3464219', '12562', '22249817', '418605', '6438', '14235691', '24235330', '1483646', '38203371', '403627', '12437', '30860403', '13457', '12388', '12796', '68300', '310782', '2839975', '1911', '19702', '4087965', '364423', '9737', '583438', '155624', '72016', '49033', '155192', '4816754', '480107', '604020', '158005', '9236'], 'Who is considered the "Father of the United States"?': ['540802', '519482', '260899', '21816397', '223202', '15124855', '261175', '1226966', '432330', '193877', '260870', '547346', '230108', '75958', '1149544', '93561', '260875', '233146', '2057489', '560404', '24905', '260898', '105892', '31171', '260873', '573097', '260910', '51591978', '261172', '427815', '261170', '21828607', '10133', '1337709', '261187', '25068298', '559128', '261182', '230337', '33299', '261168', '261185', '1943659', '27057637', '261178', '266907', '

In [12]:
import requests
from time import time
# url = 'http://35.232.59.3:8080'
# place the domain you got from ngrok or GCP IP below.
url = 'https://api.ngrok.com/reserved_domains/rd_2dKKBcXh0yNuhkFPROUvdYUUE8c'

qs_res = []
for q, true_wids in queries.items():
  duration, ap = None, None
  t_start = time()
  try:
    res = requests.get(url + '/search', {'query': q}, timeout=35)
    duration = time() - t_start
    if res.status_code == 200:
      pred_wids, _ = zip(*res.json())
      rq = results_quality(true_wids, pred_wids)
    # qs_res.append((q, duration, rq))
  except:
    pass

  qs_res.append((q, duration, rq))

In [13]:
qs_res

[('genetics', 0.23963332176208496, None),
 ('Who is considered the "Father of the United States"?',
  0.24289894104003906,
  None),
 ('economic', 0.24315190315246582, None),
 ('When was the United Nations founded?', 0.23127436637878418, None),
 ('video gaming', 0.2460782527923584, None),
 ('3D printing technology', 0.1978447437286377, None),
 ('Who is the author of "1984"?', 0.17957615852355957, None),
 ('bioinformatics', 0.19312620162963867, None),
 ('Who is known for proposing the heliocentric model of the solar system?',
  0.19271063804626465,
  None),
 ('Describe the process of water erosion.', 0.19542837142944336, None),
 ('When was the Berlin Wall constructed?', 0.19388914108276367, None),
 ('What is the meaning of the term "Habeas Corpus"?',
  0.2000141143798828,
  None),
 ('telecommunications', 0.19881033897399902, None),
 ('internet', 0.1965785026550293, None),
 ('What are the characteristics of a chemical element?',
  0.20756125450134277,
  None),
 ('Describe the structure of